In [2]:
def ner(f,f1,a1):
    from simpletransformers.ner import NERModel
    import pandas as pd
    import logging
    import re
    from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
    from transformers import pipeline
    from transformers import AutoTokenizer, AutoModelForTokenClassification
    import openpyxl
    from openpyxl import load_workbook
    logging.basicConfig(level=logging.DEBUG)
    transformers_logger = logging.getLogger('transformers')
    transformers_logger.setLevel(logging.WARNING)

    
   
    file="/host/Sahithi/myenv/input/input/"+f1
    with open(file, "r") as cur:
        text=cur.read()
        #preprocessing of data
    exceptions = {
            r'\d+mg': r'\g<0> |\g<1>',
            r'\d+:\d+': r'\g<0>',
            r'[\,\-\(\)]': r' \g<0> ',
            r'(?<!\d)\d+\.\d+(?!\d)': r'\g<0>',
            r'\d+/\d+': r'\g<0>',
            r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}': r'\g<0>',
            r'[A-Za-z]\d': r'\g<0>',
            r'[^\.\s]\.[^\.\s]': r'\g<0>',
            r'\d+\.\d+':r'\g<0>',
        }
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    processed_tokens = []

    a = ["COVID-19", "4/5Unable", "workstation:<ID", "patientÂ\x92s"]
    sentences = [] # List to store processed sentences

    for token in tokens:
        if token in a:
            processed_tokens.append(token)
        elif token not in exceptions and (":" in token or "," in token or "/" in token or "." in token or "<" in token or ">" in token or "-" in token):
            if re.match(r'\d+/\d+', token) or re.match(r'\d+:\d+', token):
                processed_tokens.append(token)
            elif re.match(r'(?<!\d)\d+\.\d+(?!\d)', token):
                processed_tokens.append(token)
            else:
                split_tokens = re.split(r'(:|,|/|\.|<|>|-)', token)
                processed_tokens.extend([t for t in split_tokens if t])
        elif token in exceptions:
            processed_tokens.append(exceptions[token])
        else:
            processed_tokens.append(token)

        # Iterate over sentences and append processed tokens
    current_sentence =""
    sentences=""
    for token in processed_tokens:
        current_sentence=current_sentence+" "+token

        if token.endswith('.') or token.endswith('!') or token.endswith('?'):
            sentences=sentences+" "+current_sentence
            current_sentence=""

    # Append the remaining tokens as the last sentence
    if current_sentence:
        sentences=sentences+current_sentence
    c=[]
        #splitting the total sentences with maximum length as 350 (can be done with any length)
    def process_input(s, max_length):
        if len(s) <= max_length:
            c.append(s)
        else:
            while len(s) > 0:
                chunk = s[:max_length] 
                c.append(chunk)
                s = s[max_length:]


    max_length = 350
    process_input(sentences, max_length)



    # Annotated file for each document
    a2="/host/Sahithi/myenv/output_entity/"+a1
    df = pd.read_csv(a2, delimiter='\n', header=None, quoting=3)
    annotated_text = df[0].tolist()
    a = []
    for i in annotated_text:
        b = i.split(' ')
        a.append(b)



    from transformers import BertTokenizer,BertForTokenClassification
    tokenizer = BertTokenizer.from_pretrained("sschet/bert-base-uncased_clinical-ner")
    model = BertForTokenClassification.from_pretrained("sschet/bert-base-uncased_clinical-ner")
    pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    x=[]
    p=[]
    for i in c: #c is the chunks of the data with max limit 350
        p.extend(pipe(i))
    #predicted ones
   
 #combining predictions dictioneries of the chunks into a list
    predicted_tokens = []
    ch=[]
    predicted_tokens=[]
    ch1=[]
    for k in c:
        ch.extend(k.split("/n")) #splitting with respect to line
    for i in ch:
        ch1.extend(i.split(" ")) #splitting with resepect to space
    while (ch1.count('')):
        ch1.remove('')
       
  
    flag=''
    for i in ch1:
        for  j in p:
            if i.lower()==j['word'] or i.lower() in j['word'] :
                predicted_tokens.append([i,j['entity_group']])
                flag='true'
                break
            else:
                flag='false'
        if flag=='false':
            predicted_tokens.append([i,'O'])


            
    annotated_tokens = [[item[0],item[3]] for item in a] 
    annotated_num=0
    for i in annotated_tokens:
        if(i[1]!='O'):
            annotated_num+=1 #counting the number of annotated tokens in the conll file


    tp = []
    tn = []
    fp = []
    fn = []
    p=predicted_tokens
    an=annotated_tokens
    for item in predicted_tokens:
        count = 0
        for items in annotated_tokens:
            if item[0] == items[0]: 

                count = 1
                if (item[1] == 'O') & (items[1]=='O'):
                    tn.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1]=="problem" or item[1]=='test' ) & (items[1]!='O'):
                    tp.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1]=="problem" or item[1]=='test') & (items[1]=='O'):
                    fp.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == 'O') & (items[1]!='O'):
                    fn.append([item,items])
                    annotated_tokens.remove(items)
                    break

        if count == 0:

            if  item[1]=='problem' or item[1]=='test':
                fp.append([[item,[item[0],'O']]])
            elif item[1] == 'O':
                tn.append([[item,[item[0],'O']]])
            
    print(len(tp),len(fp),len(tn),len(fn))
    
    inscope=0
    flag=False
    anno=0
    if(len(tp)==0 or len(fp)==0 or len(tn)==0 ):
        print("somevalue is zero")    
    else:
        
        flag=True

        total=[]
        [total.extend(l) for l in (tp,tn,fp,fn)]

        token=[]
        predict=[]
        annot=[]
        for i in total:
            if(len(i)==2):
                token.append(i[0][0])
                predict.append(i[0][1])
                annot.append(i[1][1])
            elif(len(i)==1):
                for j in i:
                    token.append(j[0][0])
                    predict.append(j[0][1])
                    annot.append(j[1][1])

        output=pd.DataFrame({
                        'token':token,
                        'predictions':predict,
                        'annotated':annot,
                    })
        

        print("annotated_num:",annotated_num)

        
        for i in total:
            if(len(i)==2):
                if(i[0][1]!='O' and i[1][1]!='O'):
                    inscope=inscope+1
                    print(i)
            elif len(i)==1:
                for j in i:
                    if(j[0][1]!='O' and j[1][1]!='O'):
                        inscope=inscope+1
                        print(j)
        print("inscope:",inscope)
        
            
        
        
    return len(token),annotated_num,inscope,flag,len(tp),len(tn),len(fp),len(fn)

    

import os
files = os.listdir('/host/Sahithi/myenv/input/input/')
an_files=os.listdir('/host/Sahithi/myenv/output_entity/')
tokens=0
annotated_tokens=0
num_file=0
inscope_tokens=0
tp=0
tn=0
fp=0
fn=0
for an_file in an_files:
    f=os.path.splitext(os.path.basename(an_file))[0]
    for file in files:
        f1=os.path.splitext(os.path.basename(file))[0]
        if f1 in f:
            print(f1,f)
            print(file)
            if file!=".ipynb_checkpoints":
                t,an,ins,flag,t1,t2,t3,t4=ner(f1,file,an_file)
                if flag==True:
                    tokens=tokens+t
                    annotated_tokens+=an
                    tp=tp+t1
                    tn=tn+t2
                    fp=fp+t3
                    fn=fn+t4
                    inscope_tokens+=ins
                    num_file+=1
                
                           
            
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall)/(precision+recall)  

    


batch_04_00011 annotator2_batch_04_00011_correct
batch_04_00011.txt


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/vocab.txt HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/vocab.txt HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/config.json HTTP/1.1" 200 0


1 112 284 0
annotated_num: 1
[['MS', 'problem'], ['MS', 'B-Disease_Subtype']]
inscope: 1
batch_05_00176 annotator2_batch_05_00176_correct
batch_05_00176.txt


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/vocab.txt HTTP/1.1" 200 0


6 167 310 4
annotated_num: 10
[['Multiple', 'problem'], ['Multiple', 'I-Disease_Subtype']]
[['Sclerosis', 'problem'], ['Sclerosis', 'I-Disease_Subtype']]
[['MS', 'problem'], ['MS', 'B-Disease_Subtype']]
[['relapse', 'problem'], ['relapse', 'B-Disease_Subtype']]
[['relapse', 'problem'], ['relapse', 'B-Relapse_Temporality']]
[['MS', 'problem'], ['MS', 'I-Disease_Subtype']]
inscope: 6
batch_05_00177 annotator2_batch_05_00177_correct
batch_05_00177.txt


...

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/vocab.txt HTTP/1.1" 200 0


5 680 801 4
annotated_num: 9
[['MS', 'problem'], ['MS', 'B-RRMS']]
[['EDSS', 'test'], ['EDSS', 'B-EDSS']]
[['-', 'problem'], ['-', 'I-RRMS']]
[['multiple', 'problem'], ['multiple', 'I-RRMS']]
[['sclerosis', 'problem'], ['sclerosis', 'I-RRMS']]
inscope: 5
recent_00050 annotator2_recent_00050_correct
recent_00050.txt


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sschet/bert-base-uncased_clinical-ner/resolve/main/config.json HTTP/1.1" 200 0


12 636 1104 5
annotated_num: 20
[['multiple', 'problem'], ['multiple', 'B-RRMS']]
[['sclerosis', 'problem'], ['sclerosis', 'I-RRMS']]
[['RRMS', 'problem'], ['RRMS', 'B-RRMS']]
[['MS', 'problem'], ['MS', 'B-RRMS']]
[['attacks', 'problem'], ['attacks', 'B-Temporal_history_of_relapse']]
[['EDSS', 'test'], ['EDSS', 'B-EDSS']]
[['EDSS', 'test'], ['EDSS', 'B-EDSS']]
[['Multiple', 'problem'], ['Multiple', 'B-RRMS']]
[['Sclerosis', 'problem'], ['Sclerosis', 'I-RRMS']]
[['RRMS', 'problem'], ['RRMS', 'B-RRMS']]
[['EDSS', 'test'], ['EDSS', 'B-EDSS']]
[['MS', 'problem'], ['MS', 'B-RRMS']]
inscope: 12
.ipynb_checkpoints .ipynb_checkpoints
.ipynb_checkpoints


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:275                                                                                  │
│ in ner:18                                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IsADirectoryError: [Errno 21] Is a directory: '/host/Sahithi/myenv/input/input/.ipynb_checkpoints'

In [3]:
print(num_file,predicted_annotated_tokens,inscope_tokens)

452 5504 3387


In [4]:
print("number of files:",num_file)
print("number of  tokens:",tokens)
print("number of annotated tokens:",annotated_tokens)
print("inscope _tokens:",inscope_tokens)
print("true positives:",tp)
print("true negatives:",tn)
print("false positives:",fp)
print("false negatives:",fn)


precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall)/(precision+recall)  

print("precision:",precision*100)
print("recall:",recall*100)
print("f1 score:",f1*100)

number of files: 452
number of  tokens: 345895
number of annotated tokens: 0
inscope _tokens: 3387
true positives: 3387
true negatives: 208482
false positives: 132244
false negatives: 1782
precision: 2.497216712993342
recall: 65.52524666279744
f1 score: 4.811079545454545
